<a href="https://colab.research.google.com/github/onpk/Science-Fair-2024/blob/main/Science_Fair_Draft_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Predicting the prescense of e-coli given data, using Computer Vision and CNNs

Use Image Segmentation to isolate the bacteria in images where there are other types of bacteria

If possible, use transfer learning to predict if a sample does not have bacteria in it

In [ ]:
#Step one: Data Preprocessing and cleaning
#Import necessary modules:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np
from google.colab import drive
import os
#from sklearn.model_selection import train_test_split

In [ ]:
#Open Data(Have seperate folders for e-coli yes and no)
from PIL import Image,ImageEnhance #Just to test if the images can open
import random
directory="/content/drive/My Drive/NR_Science_Fair_2024/Positive"
#drive.mount("/content/drive")
os.chdir(directory)
poslist={}
neglist={}
direct=os.listdir("/content/drive/My Drive/NR_Science_Fair_2024/Positive")
for d in direct:
  #temp=[]
  img=Image.open(d)
  img=img.convert("L")
  temp.append(np.asarray(img))
  temp.append("Positive")
  poslist.append(temp)
os.chdir("/content/drive/My Drive/NR_Science_Fair_2024/Negative")
direct=os.listdir("/content/drive/My Drive/NR_Science_Fair_2024/Negative")
for e in direct:
  temp=[]
  img=Image.open(e)
  img=img.convert("L")
  temp.append(np.asarray(img))
  temp.append("Negative")
  neglist.append(temp)
#img=plt.imread(direct[0])
data=poslist+neglist
random.shuffle(data)
#data=np.asarray(data)
print(data)

In [ ]:
#Getting training and Testing Data (loop through the data and randomly pick 80% of both datasets to put into the training, and put the rest into testing)
nparray=np.array(data,dtype=object)
t_temp,v_temp=np.split(nparray,[int(len(nparray)*0.8)])
t=[]
v=[]
t_labels=[]
v_labels=[]
for i in t_temp:
  t.append(i[0])
  t_labels.append(i[1])
for j in v_temp:
  v.append(j[0])
  v_labels.append(j[1])
t=np.array(t,dtype=object)
v=np.array(v,dtype=object)
print(t[1])
'''print(len(data))
flat_list = [item for sublist in data for item in sublist]
flat_array = np.array(flat_list)

# Calculate the index for splitting the array in an 80:20 ratio
split_index = int(0.8 * len(flat_array))

# Split the flat array into two arrays based on the 80:20 ratio
t = flat_array[:split_index]
v = flat_array[split_index:]
print(v[1])'''

[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]


'print(len(data))\nflat_list = [item for sublist in data for item in sublist]\nflat_array = np.array(flat_list)\n\n# Calculate the index for splitting the array in an 80:20 ratio\nsplit_index = int(0.8 * len(flat_array))\n\n# Split the flat array into two arrays based on the 80:20 ratio\nt = flat_array[:split_index]\nv = flat_array[split_index:]\nprint(v[1])'

In [ ]:
#Formatting the training and testing data assuming that the training data is put into the variable t, and the testing data is put into the variable v

# Normalize the data

flat_t=t.flatten()
mean=np.mean(flat_t)
stdev=np.std(flat_t)
flat_t=(flat_t-mean)/stdev
t_normalized=flat_t.reshape(t.shape)
flat_v=v.flatten()
mean=np.mean(flat_v)
stdev=np.std(flat_v)
flat_v=(flat_v-mean)/stdev
v_normalized=flat_v.reshape(v.shape)
'''row_sums = t.sum(axis=1)
t_normalized = t / row_sums[:, np.newaxis]

row_sums_2 = v.sum(axis=1)
v_normalized = v / row_sums[:, np.newaxis]'''
#t_normalized= t.astype(np.float32) / 255.0
#v_normalized = v.astype(np.float32) / 255.0

# Create a dummy structure to feed into ImageDataGenerator
# This step is more conceptual; in a real-world scenario, you'd likely use directories.
train_dummy_labels = np.zeros((t.shape[0],))  # Placeholder labels (assuming binary classification)
test_dummy_labels = np.zeros((v.shape[0],))    # Placeholder labels

# Create an ImageDataGenerator instance for training
train_datagen = ImageDataGenerator(
    rescale=1./255,          # normalize pixel values to [0,1]
    rotation_range=40,       # randomly rotate images in the range (degrees, 0 to 180)
    width_shift_range=0.2,   # randomly shift images horizontally (fraction of total width)
    height_shift_range=0.2,  # randomly shift images vertically (fraction of total height)
    shear_range=0.2,         # set range for random shear
    zoom_range=0.2,          # set range for random zoom
    horizontal_flip=True,    # randomly flip images
    fill_mode='nearest'      # fill in newly created pixels
)
train_generator = train_datagen.flow(t_normalized, train_dummy_labels, batch_size=32)

# Create an ImageDataGenerator instance for validation
test_datagen = ImageDataGenerator(rescale=1./255)  # Rescale for normalization (as v_normalized is already normalized)
test_generator = test_datagen.flow(v_normalized, test_dummy_labels, batch_size=32)


ValueError: ignored

In [ ]:
#Step two: Actual CNN model

# Define the model
model = Sequential()

# First convolutional layer
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 3)))
model.add(MaxPooling2D(2, 2))

# Second convolutional layer
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(2, 2))

# Third convolutional layer
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D(2, 2))

# Flatten the output of the convolutional layers
model.add(Flatten())

# Dense layers for classification
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.5))  # Dropout for regularization
model.add(Dense(1, activation='sigmoid'))  # Sigmoid activation for binary classification

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Assuming you have a dataset with train_data, train_labels, test_data, test_labels
# train_data and test_data should be in shape (num_samples, 150, 150, 3)
# train_labels and test_labels should be binary labels (0 or 1)

# Train the model
model.fit(
    train_generator,
    steps_per_epoch=len(t) // 32,  # total_train_samples should be the total number of training samples
    epochs=20,
    validation_data=test_generator,
    validation_steps=len(v) // 32  # total_validation_samples should be the total number of validation samples
)

# Evaluate the model
test_loss, test_acc = model.evaluate(test_data, test_labels)
print("Accuracy"+str(test_acc))

